In [4]:
import re
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import umap
import nltk
import spacy
import string

import matplotlib.pyplot as plt
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')
import seaborn as sns
sns.set(style='white', rc={'figure.figsize':(10,8)})

np.random.seed(42)
%config Completer.use_jedi = False
pd.set_option('display.max_columns', 30)
# pd.set_option('display.max_rows', 100)

spaci = spacy.load("en_core_web_sm")

In [5]:
df = pd.read_csv('mindvalley_fb_comments.csv')
df.head()

,Unnamed: 0,comment
0,0,After this class you'll see why Silicon Valley...
1,1,The shirt is almost identical to my moms fancy...
2,2,"When you buy a program, can you stop, backup,..."
3,3,Exactly what is lucid dreaming? Thank you for ...
4,4,"I have lucid dreams all the time, also have ov..."


In [6]:
df.comment.replace("\S*http\S*" , "", regex=True, inplace = True)
df.comment.replace("_" , " ", regex=True, inplace = True)
df.comment.replace("^" , " ", regex=True, inplace = True)

In [7]:
for i in string.punctuation:
    try:
        df.comment.replace(f"\{i}" , "", regex=True, inplace = True)
    except: pass

In [8]:
def remove_unicode(text):

    try:
        text = text.encode("latin_1").decode("raw_unicode_escape").encode('utf-16', 'surrogatepass').decode('utf-16')
    except:
        text = text

    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", re.UNICODE)

    text = re.sub(emoji_pattern, '', str(text))
    text = ''.join(text.split('\\n')) # to remove uncompiled \n
    text = re.sub('\n', '', text)     # to remove compiled \n

    return text

In [9]:
def word_root(text):
    list_data3=[]
    text = remove_unicode(text)
    doc = spaci(text)
    for token in doc:
        if token.text.isalpha():
            lemma = token.lemma_
            list_data3.append(lemma)
        else:pass
    return " ".join(list_data3)
df.comment = df.comment.apply(lambda x: word_root(str(x).lower()))  

In [10]:
"mahmut.".isalpha()

False

In [11]:
df.comment[0]

'after this class you ll see why silicon valley company be flock to trainer like charlie to be the first to start utilise this unexplored frontier of personal growth check out this link to get start sweet dream'

In [12]:
df_clean = df.comment
df_clean.shape

(1244,)

In [13]:
df_clean = df_clean.dropna()
df_clean.shape

(1244,)

In [14]:
list_data = df_clean.to_list()
list_data = [str(sent) for sent in list_data]

In [15]:
from sentence_transformers import SentenceTransformer

In [16]:
# distilbert-base-nli-mean-tokens
# paraphrase-distilroberta-base-v1

In [17]:
model=SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings1 = model.encode(list_data, show_progress_bar=True)

Batches:   0%|          | 0/39 [00:00<?, ?it/s]

In [18]:
model=SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')
embeddings2 = model.encode(list_data, show_progress_bar=True)

Batches:   0%|          | 0/39 [00:00<?, ?it/s]

In [19]:
model=SentenceTransformer('roberta-large-nli-stsb-mean-tokens')
embeddings3 = model.encode(list_data, show_progress_bar=True)

Batches:   0%|          | 0/39 [00:00<?, ?it/s]

In [20]:
model=SentenceTransformer('allenai-specter')
embeddings4 = model.encode(list_data, show_progress_bar=True)

Batches:   0%|          | 0/39 [00:00<?, ?it/s]

In [21]:
# UMAP_bert1 = umap.UMAP(n_components=5).fit_transform(embeddings1)

# UMAP_bert2 = umap.UMAP(n_components=5).fit_transform(embeddings2)

UMAP_bert3 = umap.UMAP(n_components=5).fit_transform(embeddings3)

# UMAP_bert4 = umap.UMAP(n_components=5).fit_transform(embeddings4)

#n_neighbors=10, min_dist=0.0, 

In [22]:
from pyclustertend import hopkins

embeddings = {"UMAP_bert1" : UMAP_bert1, "UMAP_bert2" : UMAP_bert2,
              "UMAP_bert3" : UMAP_bert3, "UMAP_bert4" : UMAP_bert4}
              
embeds = {}

for key, emb in embeddings.items():
    embeds[key] = hopkins(emb,emb.shape[0])

embeds
# embeddings[embeds.index(min(embeds))]
# print(f"the hopkins score for embeddings {i} is : {hs}")

NameError: name 'UMAP_bert1' is not defined

In [ ]:
best_embed_name = min(embeds, key=embeds.get)
best_embed_name = 'UMAP_bert3'
best_model = embeddings[best_embed_name]
best_embed_name

In [ ]:
# #inertia

# kmeans_per_k = [KMeans(n_clusters=k, random_state=42).fit(best_model)
#                 for k in range(1, 10)]
# inertias = [model.inertia_ for model in kmeans_per_k]

# plt.plot(range(1, 10), inertias, "bo-")
# plt.xlabel("$k$", fontsize=14)
# plt.ylabel("Inertia", fontsize=14)
# plt.show()

In [ ]:
# # #deleting junk clusters
# corpus = pd.DataFrame(df_clean)
# corpus["cluster"] = cluster
# indexes = list(corpus[corpus.cluster == 1].index) + list(corpus[corpus.cluster == 2].index) + list(corpus[corpus.cluster == 3].index)
# best_model = np.delete(best_model, indexes, 0)
# corpus = corpus.drop(axis=0, index=indexes)

In [ ]:
cluster = KMeans(n_clusters=3,init="k-means++").fit_predict(best_model)

In [ ]:
def plot(cluster = cluster):    

    clustered = (cluster >= 0)
    reduced_dim = umap.UMAP(n_components=2).fit_transform(embeddings3)



    unique, counts = np.unique(cluster, return_counts=True)
    print(dict(zip(unique, counts)))

    plt.scatter(reduced_dim[~clustered, 0],
                reduced_dim[~clustered, 1],
                c=(0.5,0.5,0.5),
                # s=1,
                alpha=0.5
                )
    plt.scatter(reduced_dim[clustered, 0],
                reduced_dim[clustered, 1],
                c=cluster[clustered],
                # s=1,
                cmap='Spectral')
    plt.show()

In [ ]:
plot()

In [ ]:
docs2 = pd.DataFrame(corpus.comment)
docs2["cluster"] = cluster
labeled_docs = docs2.groupby(["cluster"], as_index=False).agg({"comment": " ".join})
labeled_docs.comment.replace(r"\S*clas\S*", "class", regex=True, inplace = True)
labeled_docs.comment.replace(r"\S*teach\S*", "teach", regex=True, inplace = True)
labeled_docs

In [ ]:
from keybert import KeyBERT
array_text = labeled_docs.comment.tolist()

model_names = {"model1":'distilbert-base-nli-mean-tokens',
               "model2": "xlm-r-distilroberta-base-paraphrase-v1" ,
               "model3": 'roberta-large-nli-stsb-mean-tokens',
               "model4": "allenai-specter"}
    
kw_extractor = KeyBERT('xlm-r-distilroberta-base-paraphrase-v1')
k = 0
for j in range(len(array_text)):
    k+=1
    keywords = kw_extractor.extract_keywords(array_text[j],top_n=10,stop_words=["pitbullgastrobarukrainemillionariopooreameruca",
                                                                                "enseñanza",
                                                                                "vishenthank",
                                                                                "lisa",
                                                                                "ooaasmamn",
                                                                                "aprendizaje"],)
    if not keywords:
        k -= 1
    else:
        print("Cluster", str(k))
        print([word[0] for word in keywords])

In [ ]:
kw_extractor = KeyBERT('xlm-r-distilroberta-base-paraphrase-v1')
k = 0
for j in range(len(array_text)):
    k+=1
    keywords = kw_extractor.extract_keywords(array_text[j],top_n=10,keyphrase_ngram_range=(1, 2),
                                             stop_words=["pitbullgastrobarukrainemillionariopooreameruca",
                                                                                "enseñanza",
                                                                                "vishenthank",
                                                                                "lisa",
                                                                                "ooaasmamn",
                                                                                "aprendizaje"],)
    if not keywords:
        k -= 1
    else:
        print("Cluster", str(k))
        print([word[0] for word in keywords])

In [ ]:
kw_extractor = KeyBERT('xlm-r-distilroberta-base-paraphrase-v1')
k = 0
for j in range(len(array_text)):
    k+=1
    keywords = kw_extractor.extract_keywords(array_text[j],top_n=10,keyphrase_ngram_range=(1, 3),
                                             stop_words=["pitbullgastrobarukrainemillionariopooreameruca",
                                                                                "enseñanza",
                                                                                "vishenthank",
                                                                                "lisa",
                                                                                "ooaasmamn",
                                                                                "aprendizaje"],)
    if not keywords:
        k -= 1
    else:
        print("Cluster", str(k))
        print([word[0] for word in keywords])